# ML-Olympiad-can-you-guess-the-beer-style

## Import Modules

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
import pandas as pd

## Prepare Data
Load data from csv format into a dataframe.

In [24]:
df_train: pd.DataFrame = pd.read_csv("ML-Olympiad-can-you-guess-the-beer-style/train.csv")
df_test: pd.DataFrame = pd.read_csv("ML-Olympiad-can-you-guess-the-beer-style/test.csv")
df_train

,ID,Name,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews,Style
0,282,Original,Jacob Leinenkugel Brewing Company,Jacob Leinenkugel Brewing Company Leinenkugel'...,Notes:,4.6,8.0,18.0,17.0,28.0,...,42.0,3.0,43.0,2.742424,3.060606,2.960606,2.978788,3.275758,165.0,6
1,183,Lucifer,Brouwerij Het Anker,Brouwerij Het Anker Lucifer,Notes:,8.0,20.0,NaN,10.0,24.0,...,9876.0,15.0,31.0,3.776119,4.029851,3.768657,3.791045,3.843284,67.0,0
2,139,Victor Ale,Allagash Brewing Company,Allagash Brewing Company Allagash Victor Ale,Notes:The Victor Ale was inspired by its femin...,9.0,20.0,9876.0,24.0,35.0,...,26.0,15.0,39.0,3.865079,3.924603,3.960317,3.884921,3.813492,126.0,0
3,179,Budweiser,Anheuser-Busch,Anheuser-Busch Budweiser,Notes:Brewed using a blend of imported and cla...,5.0,8.0,18.0,6.0,17.0,...,15.0,3.0,50.0,2.189644,2.516476,2.473100,2.420309,2.883995,1487.0,6
4,463,Pilsener,Browerij Nacional Balashi,Browerij Nacional Balashi Balashi,Notes:The beer has a deep golden color totally...,5.0,18.0,25.0,26.0,23.0,...,21.0,1.0,50.0,2.634921,2.976190,3.055556,2.952381,3.333333,63.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,171,Tecate,"Cervecería Cuauhtémoc Moctezuma, S.A. de C.V.","Cervecería Cuauhtémoc Moctezuma, S.A. de C.V. ...",Notes:A well balanced bright golden lager beer...,4.5,NaN,18.0,23.0,14.0,...,17.0,2.0,24.0,2.388660,2.732990,2.709278,2.591753,2.952577,485.0,6
514,206,Winter Ale,Weyerbacher Brewing Co.,Weyerbacher Brewing Co. Winter Ale,Notes:This beer is not spiced.\t,5.6,15.0,25.0,14.0,38.0,...,16.0,22.0,86.0,3.418960,3.726300,NaN,3.414373,3.525994,9876.0,4
515,370,Schneider Weisse Tap 4 Wiesen Edel-Weisse,Weisses Bräuhaus G. Schneider & Sohn GmbH,Weisses Bräuhaus G. Schneider & Sohn GmbH Schn...,Notes:Also known as Tap 4 - Mein Grünes and Ta...,6.2,10.0,15.0,20.0,31.0,...,34.0,38.0,58.0,4.039030,3.914557,3.947257,4.083333,4.123418,474.0,14
516,535,Grand Cru,Allagash Brewing Company,Allagash Brewing Company Allagash Grand Cru,Notes:Grand Cru is brewed with a touch of peat...,7.2,25.0,50.0,8.0,55.0,...,29.0,64.0,70.0,4.062118,NaN,4.061100,4.124236,4.085540,491.0,7


Transform columns of dataframes to usable formats for the model.

In [17]:
def transform_dtypes(dataframe: pd.DataFrame) -> pd.DataFrame:
    dtype_dict: dict = {'ID':pd.CategoricalDtype, 
                        'Name': pd.StringDtype, 
                        'Brewery': pd.CategoricalDtype, 
                        'Beer Name (Full)': pd.StringDtype, 
                        'Description': pd.StringDtype, 
                        'ABV': pd.Float32Dtype,
                        'Min IBU': pd.Float32Dtype,
                        'Max IBU': pd.Float32Dtype,
                        'Astringency': pd.Int16Dtype, 
                        'Body': pd.Int16Dtype, 
                        'Alcohol': pd.Float32Dtype, 
                        'Bitter': pd.Int16Dtype,
                        'Sweet': pd.Int16Dtype, 
                        'Sour': pd.Int16Dtype,
                        'Salty': pd.Int16Dtype,
                        'Fruits': pd.Int16Dtype, 
                        'Hoppy': pd.Int16Dtype, 
                        'Spices': pd.Int16Dtype, 
                        'Malty': pd.Int16Dtype,
                        'review_aroma': pd.Float32Dtype,
                        'review_appearance': pd.Float32Dtype,
                        'review_palate': pd.Float32Dtype, 
                        'review_taste': pd.Float32Dtype,
                        'review_overall': pd.Float32Dtype, 
                        'number_of_reviews':pd.Int32Dtype, 
                        'Style': pd.Int16Dtype}
    return dataframe.convert_dtypes(dtype_dict)

In [18]:
df_train = transform_dtypes(dataframe = df_train)
df_test = transform_dtypes(dataframe = df_test)

Transform data types like categorical to numerics

Normalize data such that values are between 0 and 1 for loating values